In [180]:
import langchain
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from langchain.tools import StructuredTool
from pydantic import BaseModel, field_validator, ValidationError

# Setting up debugging
langchain.debug = True

In [181]:
# Define the system message
system_message = SystemMessage(content='''Act as a helpful AI interview assistant, 
                               Always ask a follow-up question, especially after returning results.
                               Ask one question at a time.
                               Always be in interview mode or tool mode.
                               Ask the user about the desired topic.
                               ''')

# Initialize memory
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [239]:
# Define Tools
def google_search(query: str):
    """Mock Google search tool"""
    return f"Found many results for {query}. Here's a link to one of them: [link]"

def save_interview(interview_text: str):
    """Mock tool to save an interview"""
    return f"Interview saved! Content: {interview_text}"

def get_pokemon(pokemon_name: str):
    """Mock tool to get a Pokémon's information"""
    return f"Found Pokémon: {pokemon_name}. Type: Electric. Ability: Static."

In [240]:
# Convert functions to Structured Tools
google_search = StructuredTool.from_function(google_search)
save_interview = StructuredTool.from_function(save_interview)
pokemon = StructuredTool.from_function(get_pokemon)

In [242]:
from typing import Optional, Type
from langchain.tools import BaseTool
from langchain.callbacks.manager import (CallbackManagerForToolRun)
from langchain.tools.base import ToolException

class FileType(BaseModel):
    file_type: str

    @field_validator("file_type")
    def check_file_type(cls, value):
        if value not in ["pdf", "docx", "doc", "txt"]:
            raise ValueError("File type must be one of: pdf, docx, doc, txt")
        return value

class ArgumentType(BaseModel):
    url: str
    file_type: FileType

class SummarizeFileFromURL(BaseTool):
    name = "SummarizeFileFromURL"
    description = "Summarize a file from a URL."
    args_schema: Type[ArgumentType] = ArgumentType

    def _run(
        self,
        url: str,
        file_type: Optional[FileType],
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return "The file contains lots of information about Pokemon, including Pikachu, Charmander, and Squirtle!"

    async def _arun(
        self,
        url: str,
        file_type: FileType,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This tool does not support async")


In [243]:
def handle_tool_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool or try again later.")

In [232]:
file_summarizer = SummarizeFileFromURL()
file_summarizer.handle_tool_error = handle_tool_error

In [244]:
# Initialize the agent with tools:
llm = ChatOpenAI(temperature=0)
tools = [google_search, save_interview, pokemon, file_summarizer]

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    "system_message": system_message
}

agent = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True,
    memory=memory,
    agent_kwargs=agent_kwargs,
)

In [234]:
# agent.run("My name is James, I'm a computer programmer and I like to play cosy games.")
# agent.run('What is my name and my profession?')

In [235]:
# agent.run('Can you get the latest pokemon?')

In [236]:
# agent.run("Can you summarize this URL? https://assets.pokemon.com/assets/cms2/pdf/trading-card-game/rulebook/sm7_rulebook_en.txt")

In [249]:
# TODO - Fix this:
# agent.run("Can you summarize this URL? https://assets.pokemon.com/assets/cms2/pdf/trading-card-game/rulebook/sm7_rulebook_en.docx")